# SCENIC+

### save rna counts in adata.raw and change count matrix barcode to ATAC one

In [1]:
import scanpy as sc

In [ ]:
rna = sc.read_h5ad("/multiome_adata_04072025.h5ad")

In [3]:
rna.layers

Layers with keys: counts, log_counts

In [4]:
rna.obs

,sample,dna_total_counts,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,pct_counts_mt,pct_counts_ribo,pct_counts_hb,...,leiden_1.25,leiden_1.5,leiden_2.0,leiden_5.0,leiden_10.0,leiden_20.0,celltypist_cell_label_coarse,celltypist_conf_score_coarse,celltypist_cell_label_fine,celltypist_conf_score_fine
AAACAGCCAGTTTGTG-D0,D0,8056.0,1891,7.545390,7641.0,8.941415,21.823060,13.584609,7.642979,0.0,...,4,4,4,1,9,37,B-cell lineage,0.978114,Pro-B cells,0.950723
AAACAGCCATGTTTGG-D0,D0,23901.0,2319,7.749322,10986.0,9.304468,21.354451,13.899508,9.111597,0.0,...,8,8,7,13,63,36,B-cell lineage,0.603815,Pro-B cells,0.370262
AAACAGCCATTTAAGC-D0,D0,31317.0,1667,7.419381,6636.0,8.800415,21.503918,13.803495,6.268836,0.0,...,4,4,4,1,26,134,B-cell lineage,0.951335,Pro-B cells,0.903238
AAACATGCATTGGGAG-D0,D0,1039.0,1599,7.377759,6888.0,8.837681,24.811266,18.626598,5.894309,0.0,...,4,4,4,10,19,81,B-cell lineage,0.121671,Pro-B cells,0.183371
AAACCAACAATAACCT-D0,D0,25718.0,1403,7.247081,5495.0,8.611775,24.913558,18.817106,5.150136,0.0,...,4,4,4,10,8,46,B-cell lineage,0.991872,Pro-B cells,0.996793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGAAGCATGAAG-Q3,Q3,13995.0,2061,7.631432,8523.0,9.050641,21.858501,14.009152,6.112871,0.0,...,5,7,18,38,30,9,B-cell lineage,0.930010,Pro-B cells,0.984357
TTTGTGAAGCGAGTAA-Q3,Q3,34490.0,1361,7.216709,5059.0,8.529121,23.739870,17.019173,8.381103,0.0,...,10,11,8,23,14,5,B-cell lineage,0.403587,Pro-B cells,0.973642
TTTGTGGCAGCCTGCA-Q3,Q3,3673.0,1536,7.337588,6272.0,8.744010,29.432398,23.852041,6.505102,0.0,...,1,0,0,16,22,58,B-cell lineage,0.484664,Pro-B cells,0.177973
TTTGTTGGTAAACAAG-Q3,Q3,9679.0,1279,7.154615,5301.0,8.575839,35.653650,28.730429,2.886248,0.0,...,9,10,11,45,50,31,Monocytes,0.745200,Non-classical monocytes,0.041574


In [ ]:
import pandas as pd
atac_CB = pd.read_csv('/barcodes/cellranger_arc_atac.737K-arc-v1.txt.gz', header = None)
rna_CB = pd.read_csv('/barcodes/cellranger_arc_rna.737K-arc-v1.txt.gz', header = None)
rna_CB.rename(columns={0: 'rna_CB'}, inplace=True)
atac_CB.rename(columns={0: 'atac_CB'}, inplace=True)
df_merged =  pd.concat([rna_CB, atac_CB], axis=1)

In [7]:
rna.obs['base_index'] = rna.obs.index.str.split('-').str[0]
mapping_dict = pd.Series(df_merged['atac_CB'].values, index=df_merged['rna_CB']).to_dict()
rna.obs['atac_CB'] = rna.obs['base_index'].map(mapping_dict)
rna.obs['sample'] = rna.obs['sample'].str.split('_').str[0]
rna.obs['atac_CB'] = rna.obs['atac_CB'].astype(str)+'-'+rna.obs['sample'].astype(str) + '___' + rna.obs['sample'].astype(str)
rna.obs.set_index('atac_CB', inplace=True)
rna.obs

,sample,dna_total_counts,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,pct_counts_mt,pct_counts_ribo,pct_counts_hb,...,leiden_1.5,leiden_2.0,leiden_5.0,leiden_10.0,leiden_20.0,celltypist_cell_label_coarse,celltypist_conf_score_coarse,celltypist_cell_label_fine,celltypist_conf_score_fine,base_index
atac_CB,,,,,,,,,,,,,,,,,,,,,
ACAGCGGGTGTTACTA-D0___D0,D0,8056.0,1891,7.545390,7641.0,8.941415,21.823060,13.584609,7.642979,0.0,...,4,4,1,9,37,B-cell lineage,0.978114,Pro-B cells,0.950723,AAACAGCCAGTTTGTG
ACAGCGGGTTTATCCT-D0___D0,D0,23901.0,2319,7.749322,10986.0,9.304468,21.354451,13.899508,9.111597,0.0,...,8,7,13,63,36,B-cell lineage,0.603815,Pro-B cells,0.370262,AAACAGCCATGTTTGG
ACAGCGGGTTCACTTT-D0___D0,D0,31317.0,1667,7.419381,6636.0,8.800415,21.503918,13.803495,6.268836,0.0,...,4,4,1,26,134,B-cell lineage,0.951335,Pro-B cells,0.903238,AAACAGCCATTTAAGC
CATTTAGGTGTCCACA-D0___D0,D0,1039.0,1599,7.377759,6888.0,8.837681,24.811266,18.626598,5.894309,0.0,...,4,4,10,19,81,B-cell lineage,0.121671,Pro-B cells,0.183371,AAACATGCATTGGGAG
CTTTATCGTCGAGGCA-D0___D0,D0,25718.0,1403,7.247081,5495.0,8.611775,24.913558,18.817106,5.150136,0.0,...,4,4,10,8,46,B-cell lineage,0.991872,Pro-B cells,0.996793,AAACCAACAATAACCT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTAGGTGTCAGTGAAC-Q3___Q3,Q3,13995.0,2061,7.631432,8523.0,9.050641,21.858501,14.009152,6.112871,0.0,...,7,18,38,30,9,B-cell lineage,0.930010,Pro-B cells,0.984357,TTTGTGAAGCATGAAG
GTAGGTGTCGGTCCAT-Q3___Q3,Q3,34490.0,1361,7.216709,5059.0,8.529121,23.739870,17.019173,8.381103,0.0,...,11,8,23,14,5,B-cell lineage,0.403587,Pro-B cells,0.973642,TTTGTGAAGCGAGTAA
GTTTGGTGTTTCTAAG-Q3___Q3,Q3,3673.0,1536,7.337588,6272.0,8.744010,29.432398,23.852041,6.505102,0.0,...,0,0,16,22,58,B-cell lineage,0.484664,Pro-B cells,0.177973,TTTGTGGCAGCCTGCA


In [8]:
rna = rna[rna.obs['celltypist_cell_label_fine'].isin(['Pro-B cells']) & rna.obs['sample'].isin(['D0','Q1'])].copy()

In [9]:
rna.obs['celltypist_cell_label_fine'].value_counts()

Pro-B cells    5361
Name: celltypist_cell_label_fine, dtype: int64

In [10]:
rna.obs['sample'].value_counts()

Q1    3864
D0    1497
Name: sample, dtype: int64

In [11]:
rna.X = rna.layers['counts'].copy()
rna.raw = rna

In [ ]:
rna.write_h5ad("multiome_adata_04072025_raw_counts.BALL.h5ad")

# run scenicplus

In [13]:
! scenicplus


   ____   ____ _____ _   _ ___ ____      
  / ___| / ___| ____| \ | |_ _/ ___| _ 
  \___ \| |   |  _| |  \| || | |   _|.|_
   ___) | |___| |___| |\  || | |__|_..._|
  |____/ \____|_____|_| \_|___\____||_| 


scenicplus verions: 1.0a1
usage: scenicplus [-h] {init_snakemake,prepare_data,grn_inference} ...

Single-Cell Enhancer-driven gene regulatory Network Inference and Clustering

positional arguments:
  {init_snakemake,prepare_data,grn_inference}

options:
  -h, --help            show this help message and exit


In [14]:
!scenicplus init_snakemake --out_dir scplus_pipeline

2025-07-11 09:36:28,372 SCENIC+      INFO     Creating snakemake folder in: scplus_pipeline


In [15]:
!tree scplus_pipeline/

/bin/bash: line 1: tree: command not found


```
tree -d scplus_pipeline/
scplus_pipeline/
└── Snakemake
    ├── config
    └── workflow
```

In [16]:
!mkdir -p outs
!mkdir -p tmp

In [17]:
! ls

01_SCENIC_plus.ipynb
02_scenic_output.ipynb
figures
multiome_adata_04072025_raw_counts.BALL.h5ad
old
outs
scplus_pipeline
subset_concatenated_adata_log_pca_celltypist_23012025_with_raw_counts.BALL.h5ad
tmp


In [18]:
!cd scplus_pipeline/Snakemake/

In [19]:
!ls

01_SCENIC_plus.ipynb
02_scenic_output.ipynb
figures
multiome_adata_04072025_raw_counts.BALL.h5ad
old
outs
scplus_pipeline
subset_concatenated_adata_log_pca_celltypist_23012025_with_raw_counts.BALL.h5ad
tmp


In [ ]:
!cat /scplus_pipeline/Snakemake/config/config.yaml

In [ ]:
cd /scplus_pipeline/Snakemake/

/lustre1/project/stg_00096/home/projects/2023_Cools_B-ALL/scenic_plus/subset/scplus_pipeline/Snakemake


/opt/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [33]:
! snakemake --cores 20 

Assuming unrestricted shared filesystem usage for local execution.
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 20
Rules claiming more threads will be scaled down.
Job stats:
job                           count
--------------------------  -------
AUCell_direct                     1
AUCell_extended                   1
all                               1
eGRN_direct                       1
eGRN_extended                     1
get_search_space                  1
motif_enrichment_cistarget        1
motif_enrichment_dem              1
prepare_GEX_ACC_multiome          1
prepare_menr                      1
region_to_gene                    1
scplus_mudata                     1
tf_to_gene                        1
total                            13

Select jobs to execute...
Execute 1 jobs...

[Fri Jul 11 11:20:58 2025]
localrule motif_enrichment_cistarget:
    input: /staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/atac/outs/region_sets, /staging/leuven/stg_00096

In [34]:
! cat .snakemake/log/2025-07-11T112057.951245.snakemake.log

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 20
Rules claiming more threads will be scaled down.
Job stats:
job                           count
--------------------------  -------
AUCell_direct                     1
AUCell_extended                   1
all                               1
eGRN_direct                       1
eGRN_extended                     1
get_search_space                  1
motif_enrichment_cistarget        1
motif_enrichment_dem              1
prepare_GEX_ACC_multiome          1
prepare_menr                      1
region_to_gene                    1
scplus_mudata                     1
tf_to_gene                        1
total                            13

Select jobs to execute...
Execute 1 jobs...

[Fri Jul 11 11:20:58 2025]
localrule motif_enrichment_cistarget:
    input: /staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/atac/outs/region_sets, /staging/leuven/stg_00096/home/projects/2023_Cools_B-ALL/scenic_plus/10x_ONT_BALL.regions_vs

In [ ]:
pip list

Package                              Version
------------------------------------ -----------------
adjustText                           1.0.4
aiohttp                              3.9.3
aiosignal                            1.3.1
anndata                              0.10.5.post1
annoy                                1.17.3
appdirs                              1.4.4
arboreto                             0.1.6
argparse-dataclass                   2.0.0
array_api_compat                     1.5.1
asttokens                            2.4.1
attr                                 0.3.2
attrs                                23.2.0
bbknn                                1.6.0
beautifulsoup4                       4.12.3
bidict                               0.23.1
bioservices                          1.11.2
blosc2                               2.5.1
bokeh                                3.4.0
boltons                              23.1.1
bs4                                  0.0.2
cattrs                     